<a href="https://colab.research.google.com/github/icarocarmona/pos-tech-dtat/blob/main/f3_big_data/Spark_FRAMEWORK_DE_BIG_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=e33945306eff8341d77ddb56c1c93f2954ab7b3e15676ce73870944f45b459fb
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


Criando a sessão do SparkContext e SparkSession

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext.getOrCreate()

In [4]:
spark = SparkSession.builder.appName('DataFrame e RDD').getOrCreate()

In [5]:
# create pyspark dataframe from an existing rdd

rdd = sc.parallelize([("C",85,76,87,91),("A",35,46,67,91),("B",87,86,37,94),("A",15,63,75,16)], 4)

In [6]:
print(type(rdd))

<class 'pyspark.rdd.RDD'>


In [7]:
sub = ['id_person', 'value_1', 'value_2', 'value_3', 'value_4']

In [8]:
marks_df = spark.createDataFrame(rdd, schema=sub)

In [9]:
print(type(marks_df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [10]:
marks_df.printSchema()

root
 |-- id_person: string (nullable = true)
 |-- value_1: long (nullable = true)
 |-- value_2: long (nullable = true)
 |-- value_3: long (nullable = true)
 |-- value_4: long (nullable = true)



In [11]:
marks_df.show()

+---------+-------+-------+-------+-------+
|id_person|value_1|value_2|value_3|value_4|
+---------+-------+-------+-------+-------+
|        C|     85|     76|     87|     91|
|        A|     35|     46|     67|     91|
|        B|     87|     86|     37|     94|
|        A|     15|     63|     75|     16|
+---------+-------+-------+-------+-------+



In [12]:
df = spark.read.csv('cereal.csv', sep=',', inferSchema=True, header = True)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/content/cereal.csv.

In [ ]:
df.printSchema()

# Select()

In [ ]:
df.select('Cereal Name','Manufacturer', 'Type' ).show()

# withColumn()

In [ ]:
df.withColumn('Name', df['Cereal Name'].cast("String")).printSchema()

# groupBy

In [ ]:
df.groupBy('Cereal Name', 'Calories').count().show()

# orderBy()

In [ ]:
df.orderBy('Protein (g)').show()

# Case When

In [ ]:
from pyspark.sql.functions import when

In [ ]:
df.select("Cereal Name", when(df['Vitamins and Minerals'] >= "25", "rich in vitamins").alias('rich in vitamins')).show()

# filter()

In [ ]:
df.filter(df['Calories'] == '100').show()

# isnull isnotnull

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df.filter(df['Cereal Name'].isNotNull()).show()

In [ ]:
df.filter(df['Cereal Name'].isNull()).show()

In [ ]:
df.filter(df['Cereal Name'].isNull()).count()

# Spark SQL Consultas e Seleções

In [ ]:
spark = SparkSession.builder.master("local[*]").appName('Spark SQL').getOrCreate()

In [ ]:
df = spark.sql("select 'OK' as Status")

In [ ]:
df.show()

In [ ]:
df = spark.read.csv('cereal.csv', sep=',', inferSchema=True, header = True)

In [ ]:
df.show(5)

In [ ]:
df.createOrReplaceTempView("cereal")

In [ ]:
cereal = spark.sql("SELECT * FROM cereal where Type = 'C'")
cereal.show(5)

In [ ]:
df.where(df['Type'] == 'C').show(2)

In [ ]:
spark.sql("SELECT Manufacturer FROM cereal group by Manufacturer").show()

In [ ]:
spark.sql("SELECT Count(*) as total FROM cereal where Manufacturer = 'General Mills'").show()

In [ ]:
cereal = spark.sql("SELECT 'Cereal Name' as name, Type as type, Manufacturer as mfr  FROM cereal ").show()
cereal

In [ ]:
cereal = spark.sql("SELECT 'Cereal Name' as name, Type as type , Manufacturer as mfr  FROM cereal ").show()
cereal

In [ ]:
cereal = spark.sql("SELECT DISTINCT  Type as type , Manufacturer as mfr  FROM cereal ")
cereal.count()


In [ ]:
spark.sql("SELECT * FROM cereal LIMIT 1").show()

In [ ]:
cereal = spark.sql("SELECT  Manufacturer,  Type, COUNT(*) FROM cereal  GROUP BY Manufacturer,  Type")
cereal.show()


In [ ]:
cereal = spark.sql("SELECT  Manufacturer,  Type, COUNT(*) as total , SUM(Calories) FROM cereal  GROUP BY Manufacturer,  Type")
cereal.show()


In [ ]:
cereal = spark.sql("""

SELECT  DISTINCT Type from cereal
""")
cereal.show()


In [ ]:
cereal = spark.sql("""
SELECT  Manufacturer,
  (
    CASE
      WHEN Type = 'C' THEN 'A'
      WHEN Type = 'H' THEN 'B'
      ELSE 'C'
    END
  ) AS Type_Name,
  COUNT(*) AS Total,
  SUM(Calories) AS Total_Calories
FROM cereal
GROUP BY
  Manufacturer , Type
""")
cereal.show()

# Consultas Avançadas em Spark SQL


In [ ]:
cereal = spark.sql("""
SELECT  Manufacturer AS mfr,
  Type AS type,
  sum(Calories) AS sum_calories,
  min(Calories) AS min_calories,
  max(Calories) AS max_calories,
  avg(Calories) AS avg_calories,
  count(distinct `Cereal Name`) AS count_distinct_name,
  count(`Cereal Name`) AS count_name
FROM cereal
GROUP BY Manufacturer, Type
ORDER BY Manufacturer, Type
""")
cereal.show()

In [ ]:
cereal = spark.sql("""
SELECT  Manufacturer AS mfr,
  Type AS type,
  sum(Calories) AS sum_calories,
  min(Calories) AS min_calories,
  max(Calories) AS max_calories,
  cast(avg(Calories) as decimal(10,2)) AS avg_calories,
  count(distinct `Cereal Name`) AS count_distinct_name,
  count(`Cereal Name`) AS count_name
FROM cereal
GROUP BY Manufacturer, Type
ORDER BY Manufacturer, Type
""")
cereal.show()

In [ ]:
cereal = spark.sql("""
SELECT  Manufacturer AS mfr,
  Type AS type,

  sum(Calories) AS sum_calories,
  min(Calories) AS min_calories,
  max(Calories) AS max_calories,
  cast(avg(Calories) as decimal(10,2)) AS avg_calories,

  sum(Carbs) AS sum_carbs,
  min(Carbs) AS min_carbs,
  max(Carbs) AS max_carbs,
  cast(avg(Carbs) as decimal(10,2)) AS avg_carbs,

  sum(`Vitamins and Minerals`) AS sum_vitamins,
  min(`Vitamins and Minerals`) AS min_vitamins,
  max(`Vitamins and Minerals`) AS max_vitamins,
  cast(avg(`Vitamins and Minerals`) as decimal(10,2)) AS avg_vitamins,


  count(distinct `Cereal Name`) AS count_distinct_name,
  count(`Cereal Name`) AS count_name
FROM cereal
GROUP BY Manufacturer, Type
ORDER BY Manufacturer, Type
""")
cereal.show()


In [ ]:
cereal = spark.sql("""
SELECT  Manufacturer AS mfr,
  Type AS type,
  (
    CASE
      WHEN  substr(mfr,1,1) = 'A' then 'pera'
      WHEN  mfr = 'A' then 'pera'
      WHEN  mfr = 'A' then 'pera'
      WHEN  mfr = 'A' then 'pera'
      WHEN  mfr = 'A' then 'pera'
      WHEN  mfr = 'A' then 'pera'
      WHEN  mfr = 'A' then 'pera'
      eLSE 'NA'
    END
  ) mfr_modified,
  sum(Calories) AS sum_calories,
  min(Calories) AS min_calories,
  max(Calories) AS max_calories,
  cast(avg(Calories) as decimal(10,2)) AS avg_calories,

  sum(Carbs) AS sum_carbs,
  min(Carbs) AS min_carbs,
  max(Carbs) AS max_carbs,
  cast(avg(Carbs) as decimal(10,2)) AS avg_carbs,

  sum(`Vitamins and Minerals`) AS sum_vitamins,
  min(`Vitamins and Minerals`) AS min_vitamins,
  max(`Vitamins and Minerals`) AS max_vitamins,
  cast(avg(`Vitamins and Minerals`) as decimal(10,2)) AS avg_vitamins,


  count(distinct `Cereal Name`) AS count_distinct_name,
  count(`Cereal Name`) AS count_name
FROM cereal
GROUP BY Manufacturer, Type
ORDER BY Manufacturer, Type
""")
cereal.show()


# Transformations

In [ ]:
data = [1, 2, 3, 4,5]
rdd = sc.parallelize(data)
new_rdd = rdd.map(lambda x: x* 2)
print(new_rdd.collect())

## filter()

In [ ]:
data = [1, 2, 3, 4,5, 6, 7, 8]
rdd = sc.parallelize(data)
new_rdd = rdd.filter(lambda x: x%2 == 0)
print(new_rdd.collect())

## distinct

In [ ]:
data = [1, 2, 3, 4,5, 6, 7, 8]
rdd = sc.parallelize(data)
new_rdd = rdd.distinct()
print(new_rdd.collect())

# groupByKey

In [ ]:
rdd = sc.parallelize([('a',1),('a',2),('a',3),('b',1)])
res = rdd.groupByKey().mapValues(list)
print(res.collect())

## reduceByKey()

In [ ]:
from operator import add

rdd = sc.parallelize([('a',1),('a',2),('a',3),('b',1)])
res = rdd.reduceByKey(add)
print(res.collect())

# sortByKey() == order by

In [ ]:
rdd = sc.parallelize([('a',1),('a',2),('a',3),('b',1)])
res = rdd.sortByKey()
res.collect()

## union()

In [ ]:
rdd1 = sc.parallelize([1, 2, 3, 4,5, 6, 7, 8])
rdd2 = sc.parallelize([9, 10, 11, 12,13, 14, 15])
new_rdd = rdd1.union(rdd2)
new_rdd.collect()

# Actions

## count()

In [14]:
data = [1,1,1,4,5,6,7,8,9,  10]
myRdd = sc.parallelize(data)

myRdd.count()

10

## reduce()

In [15]:
data = [1,2,3,4,5]
myRdd = sc.parallelize(data)

myRdd.reduce(lambda x, y : x * y)

120

## foreach()

In [17]:
def fun(x):
  print(x)

data = ['Scala', 'Python', 'Java', 'R']
rdd = sc.parallelize(data)

rdd.foreach(fun)

## countByValue()

In [18]:
data = ['Scala', 'Python', 'Java', 'R', 'Python', 'Java', 'R']
rdd = sc.parallelize(data)

rdd.countByValue().items()


dict_items([('Scala', 1), ('Python', 2), ('Java', 2), ('R', 2)])

## countbyKey()

In [19]:
data = [('a',1),('b',1),('c',1),('a',1)]
rdd = sc.parallelize(data)

rdd.countByKey()

defaultdict(int, {'a': 2, 'b': 1, 'c': 1})

## take(n)

In [20]:
data = [ 2,5,3,8,4]
rdd = sc.parallelize(data)
rdd.take(3)

[2, 5, 3]

In [21]:
data = [ 2,5,3,8,4]
rdd = sc.parallelize(data)
rdd.top(3)

[8, 5, 4]

In [23]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Tutorial').getOrCreate()
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
rdd = spark.sparkContext.parallelize(dept)

In [24]:
df = rdd.toDF()
df.printSchema()
df.show(truncate=False)

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)

+---------+---+
|_1       |_2 |
+---------+---+
|Finance  |10 |
|Marketing|20 |
|Sales    |30 |
|IT       |40 |
+---------+---+



In [25]:
deptColumns = ["dept_name","dept_id"]
df2 = rdd.toDF(deptColumns)
df2.printSchema()
df2.show(truncate=False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [26]:
deptDF = spark.createDataFrame(rdd, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



In [28]:
from pyspark.sql.types import StructType,StructField, StringType
deptSchema = StructType([
    StructField('dept_name', StringType(), True),
    StructField('dept_id', StringType(), True)
])

deptDF1 = spark.createDataFrame(rdd, schema = deptSchema)
deptDF1.printSchema()
deptDF1.show(truncate=False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: string (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+

